<a href="https://colab.research.google.com/github/teteumac/Acoplamento-Quartico-Anomalo/blob/main/Binary_BDTModels.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Mon May 17 15:01:00 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [27]:
%matplotlib inline

In [28]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [29]:
from lightgbm import LGBMClassifier
from sklearn.pipeline import Pipeline


In [30]:
#!python3 -m pip install coffea mplhep
!pip install matplotlib==3.1.3

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import math
#import mplhep as hep
from pandas import set_option
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import fbeta_score
from sklearn.metrics import make_scorer
from sklearn.model_selection import KFold
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
import h5py
from sklearn.metrics import auc,make_scorer,fbeta_score,precision_score,recall_score,accuracy_score,log_loss,roc_auc_score,classification_report,f1_score,confusion_matrix,roc_curve,precision_recall_curve,average_precision_score


In [31]:
def do_scaler(X,Y, scaler_X, scaler_Y):
    if scaler_X == None:
        X_scaled = X
    else:
        X_scaled = scaler_X.fit_transform(X)
    if scaler_Y == None:
        Y_scaled = Y
    else:
        Y_scaled = scaler_Y.fit_transform(Y)
    return X_scaled, Y_scaled, scaler_X, scaler_Y

def test_model(X,Y, model, scaler_X = None, verbose = 1):
    if scaler_X is not None:
        X_scaled = scaler_X.transform(X)
    else:
        X_scaled = X
    y_probs = model.predict_proba(X_scaled)  # calculate the probability
    preds = model.predict(X_scaled)
    prec, rec, thresh = precision_recall_curve(Y, y_probs[:,1])
    bidx = np.argmax(prec * rec)
    best_cut = thresh[bidx]
    print(best_cut)

    preds = y_probs[:,1] >= best_cut

    if verbose == 1:

        fpr, tpr, thresholds = metrics.roc_curve(Y, y_probs[:,1], drop_intermediate=False)
        print(f'AUC = {metrics.auc(fpr, tpr)}')
        print("Purity in test sample     : {:2.2f}%".format(100 * precision_score(Y, preds)))
        print("Efficiency in test sample : {:2.2f}%".format(100 * recall_score(Y, preds)))
        print("Accuracy in test sample   : {:2.2f}%".format(100 * accuracy_score(Y, preds)))


In [32]:
PATH = '/content/gdrive/My Drive/Colab Notebooks/output_for_LGBM/'
#plt.style.use(hep.style.ROOT)

def open_file_MC( file ):
    df = None
    with h5py.File( file , 'r' ) as f:
        dset = f[ 'dados' ]
        array = np.array( dset ) 
        array_cut = (array[:,0] > 600) #& (array[:,1] >= 200) & (array[:,2] >= 2) & (array[:,3] >= 2) & (array[:,4] >= 200) & (array[:,5] <= 2.4) & (array[:,7] <=0.6) & (array[:,8] >= 40) & (array[:,9] >= 53)  & (array[:,10] <= 2.4)
        DataSet_ = array[array_cut]
        dataframe = pd.DataFrame( DataSet_ , columns = ['Mww', 'Pt_W_lep', 'dPhi_Whad_Wlep', 'dPhi_jatos_MET', 'jetAK8_pt','jetAK8_eta', 'jetAK8_prunedMass',
'jetAK8_tau21', 'METPt', 'muon_pt', 'muon_eta', 'ExtraTracks', 'PUWeight', 'Yww', 'xi1', 'xi2', 'Mx', 'Yx','Mww/Mx', 'Yww_Yx', 'weight'] )
        dataframe['Acoplanaridade_Whad_Wlep'] = 1 - dataframe['dPhi_Whad_Wlep'].abs()/math.pi
        dataframe['Acoplanaridade_jatos_MET'] = 1 - dataframe['dPhi_jatos_MET'].abs()/math.pi       
        return dataframe

def open_file_DD( file ):
    df = None
    with h5py.File( file , 'r' ) as f:
        dset = f[ 'dados' ]
        array = np.array( dset ) 
        array_cut = (array[:,0] > 600) #& (array[:,1] >= 200) & (array[:,2] >= 2) & (array[:,3] >= 2) & (array[:,4] >= 200) & (array[:,5] <= 2.4) & (array[:,7] <=0.6) & (array[:,8] >= 40) & (array[:,9] >= 53)  & (array[:,10] <= 2.4)
        DataSet_ = array[array_cut]        
        dataframe = pd.DataFrame( DataSet_ , columns = ['Mww', 'Pt_W_lep', 'dPhi_Whad_Wlep', 'dPhi_jatos_MET', 'jetAK8_pt',
       'jetAK8_eta', 'jetAK8_prunedMass', 'jetAK8_tau21', 'METPt', 'muon_pt',
       'muon_eta', 'ExtraTracks', 'Yww', 'xi1', 'xi2', 'MultiRP1', 'MultiRP2',
       'Mx', 'Yx', 'Mww/Mx', 'Yww_Yx', 'weight'] )
        dataframe['Acoplanaridade_Whad_Wlep'] = 1 - dataframe['dPhi_Whad_Wlep'].abs()/math.pi
        dataframe['Acoplanaridade_jatos_MET'] = 1 - dataframe['dPhi_jatos_MET'].abs()/math.pi       
        return dataframe

def open_file_Data( file ):
    df = None
    with h5py.File( file , 'r' ) as f:
        dset = f[ 'dados' ]
        array = np.array( dset ) 
        array_cut = (array[:,0] > 600) #& (array[:,1] >= 200) & (array[:,2] >= 2) & (array[:,3] >= 2) & (array[:,4] >= 200) & (array[:,5] <= 2.4) & (array[:,7] <=0.6) & (array[:,8] >= 40) & (array[:,9] >= 53)  & (array[:,10] <= 2.4)
        DataSet_ = array[array_cut]        
        dataframe = pd.DataFrame( DataSet_ , columns = ['Mww', 'Pt_W_lep', 'dPhi_Whad_Wlep', 'dPhi_jatos_MET', 'jetAK8_pt','jetAK8_eta', 'jetAK8_prunedMass',
'jetAK8_tau21', 'METPt', 'muon_pt', 'muon_eta', 'ExtraTracks', 'PUWeight', 'Yww', 'xi1', 'xi2','Mx', 'Yx', 
'Mww/Mx', 'Yww_Yx'] )
        dataframe['Acoplanaridade_Whad_Wlep'] = 1 - dataframe['dPhi_Whad_Wlep'].abs()/math.pi
        dataframe['Acoplanaridade_jatos_MET'] = 1 - dataframe['dPhi_jatos_MET'].abs()/math.pi        
        return dataframe

#select_columns = ['Mww', 'Pt_W_lep', 'jetAK8_pt','jetAK8_eta', 'jetAK8_tau21', 'METPt', 'muon_pt', 'muon_eta', 'ExtraTracks', 'Yww', 'xi1', 'xi2','Mx', 'Mww/Mx']
select_columns = ['Mww', 'Pt_W_lep', 'Acoplanaridade_jatos_MET', 'Acoplanaridade_jatos_MET', 'jetAK8_pt','jetAK8_eta', 'jetAK8_prunedMass', 'jetAK8_tau21', 'METPt', 'muon_pt', 'muon_eta', 'ExtraTracks', 'Mx', 'Mww/Mx']


SM =       open_file_MC( PATH + 'DataSet_SM_multiRP.h5')
ANOMALO1 = open_file_MC( PATH + 'DataSet_ANOMALO1_multiRP.h5')
ANOMALO2 = open_file_MC( PATH + 'DataSet_ANOMALO2_multiRP.h5')
ANOMALO3 = open_file_MC( PATH + 'DataSet_ANOMALO3_multiRP.h5')
ANOMALO4 = open_file_MC( PATH + 'DataSet_ANOMALO4_multiRP.h5')
ANOMALO5 = open_file_MC( PATH + 'DataSet_ANOMALO5_multiRP.h5')
ANOMALO6 = open_file_MC( PATH + 'DataSet_ANOMALO6_multiRP.h5')
ANOMALO7 = open_file_MC( PATH + 'DataSet_ANOMALO7_multiRP.h5')
ANOMALO8 = open_file_MC( PATH + 'DataSet_ANOMALO8_multiRP.h5')

label_signal1  = pd.DataFrame( [1]*len( ANOMALO1 ) )
label_signal2  = pd.DataFrame( [1]*len( ANOMALO2 ) )
label_signal3  = pd.DataFrame( [1]*len( ANOMALO3 ) )
label_signal4  = pd.DataFrame( [1]*len( ANOMALO4 ) )
label_signal5  = pd.DataFrame( [1]*len( ANOMALO5 ) )
label_signal6  = pd.DataFrame( [1]*len( ANOMALO6 ) )
label_signal7  = pd.DataFrame( [1]*len( ANOMALO7 ) )
label_signal8  = pd.DataFrame( [1]*len( ANOMALO8 ) )
label_signalSM = pd.DataFrame( [1]*len( SM ) )

SM = pd.concat( [ SM, label_signalSM ], axis = 1 ).rename(columns={0: 'label'})
print('Shape for Standard Model --> ', SM.shape)

ANOMALO1 = pd.concat( [ ANOMALO1, label_signal1 ], axis = 1 ).rename(columns={0: 'label'})
print('Shape for Anomalo 1 --> ', ANOMALO1.shape)

ANOMALO2 = pd.concat( [ ANOMALO2, label_signal2 ], axis = 1 ).rename(columns={0: 'label'})
print('Shape for Anomalo 2 --> ', ANOMALO2.shape)

ANOMALO3 = pd.concat( [ ANOMALO3, label_signal3 ], axis = 1 ).rename(columns={0: 'label'})
print('Shape for Anomalo 3 --> ', ANOMALO3.shape)

ANOMALO4 = pd.concat( [ ANOMALO4, label_signal4 ], axis = 1 ).rename(columns={0: 'label'})
print('Shape for Anomalo 4 --> ', ANOMALO4.shape)

ANOMALO5 = pd.concat( [ ANOMALO5, label_signal5 ], axis = 1 ).rename(columns={0: 'label'})
print('Shape for Anomalo 5 --> ', ANOMALO5.shape)

ANOMALO6 = pd.concat( [ ANOMALO6, label_signal6 ], axis = 1 ).rename(columns={0: 'label'})
print('Shape for Anomalo 6 --> ', ANOMALO6.shape)

ANOMALO7 = pd.concat( [ ANOMALO7, label_signal7 ], axis = 1 ).rename(columns={0: 'label'})
print('Shape for Anomalo 7 --> ', ANOMALO7.shape)

ANOMALO8 = pd.concat( [ ANOMALO8, label_signal8 ], axis = 1 ).rename(columns={0: 'label'})
print('Shape for Anomalo 8 --> ', ANOMALO8.shape)

data_set_back_multirp = open_file_DD( PATH + 'DataDriven_Background_multiRP.h5' )

label_back = pd.DataFrame( [0]*len( data_set_back_multirp ) )
data_set_back_multirp = pd.concat( [ data_set_back_multirp, label_back ], axis = 1 ).rename(columns={0: 'label'})

Dataset_Signal_Back1  = pd.concat( [ ANOMALO1 , data_set_back_multirp, SM  ], axis = 0, sort = False )
Dataset_Signal_Back2  = pd.concat( [ ANOMALO2 , data_set_back_multirp, SM  ], axis = 0, sort = False )
Dataset_Signal_Back3  = pd.concat( [ ANOMALO3 , data_set_back_multirp, SM  ], axis = 0, sort = False )
Dataset_Signal_Back4  = pd.concat( [ ANOMALO4 , data_set_back_multirp, SM  ], axis = 0, sort = False )
Dataset_Signal_Back5  = pd.concat( [ ANOMALO5 , data_set_back_multirp, SM  ], axis = 0, sort = False )
Dataset_Signal_Back6  = pd.concat( [ ANOMALO6 , data_set_back_multirp, SM  ], axis = 0, sort = False )
Dataset_Signal_Back7  = pd.concat( [ ANOMALO7 , data_set_back_multirp, SM  ], axis = 0, sort = False )
Dataset_Signal_Back8  = pd.concat( [ ANOMALO8 , data_set_back_multirp, SM  ], axis = 0, sort = False )

data_set_dados_multirp = open_file_Data( PATH + 'DataSet_dados_multiRP.h5' )
data_set_dados_multirp = data_set_dados_multirp[select_columns]

from sklearn.model_selection import train_test_split

test_size = 0.40
DataSet_Train1_, DataSet_Test1_ = train_test_split( Dataset_Signal_Back1, test_size = test_size, random_state = 42, stratify = Dataset_Signal_Back1.label )
DataSet_Train2_, DataSet_Test2_ = train_test_split( Dataset_Signal_Back2, test_size = test_size, random_state = 42, stratify = Dataset_Signal_Back2.label )
DataSet_Train3_, DataSet_Test3_ = train_test_split( Dataset_Signal_Back3, test_size = test_size, random_state = 42, stratify = Dataset_Signal_Back3.label )
DataSet_Train4_, DataSet_Test4_ = train_test_split( Dataset_Signal_Back4, test_size = test_size, random_state = 42, stratify = Dataset_Signal_Back4.label )
DataSet_Train5_, DataSet_Test5_ = train_test_split( Dataset_Signal_Back5, test_size = test_size, random_state = 42, stratify = Dataset_Signal_Back5.label )
DataSet_Train6_, DataSet_Test6_ = train_test_split( Dataset_Signal_Back6, test_size = test_size, random_state = 42, stratify = Dataset_Signal_Back6.label )
DataSet_Train7_, DataSet_Test7_ = train_test_split( Dataset_Signal_Back7, test_size = test_size, random_state = 42, stratify = Dataset_Signal_Back7.label )
DataSet_Train8_, DataSet_Test8_ = train_test_split( Dataset_Signal_Back8, test_size = test_size, random_state = 42, stratify = Dataset_Signal_Back8.label )


DataSet_Train8 = DataSet_Train8_[select_columns] 
DataSet_Test8 = DataSet_Test8_[select_columns] 

DataSet_Train7 = DataSet_Train7_[select_columns] 
DataSet_Test7 = DataSet_Test7_[select_columns] 

DataSet_Train6 = DataSet_Train6_[select_columns] 
DataSet_Test6 = DataSet_Test6_[select_columns] 

DataSet_Train5 = DataSet_Train5_[select_columns] 
DataSet_Test5 = DataSet_Test5_[select_columns] 

DataSet_Train1 = DataSet_Train1_[select_columns] 
DataSet_Test1 = DataSet_Test1_[select_columns] 

DataSet_Train2 = DataSet_Train2_[select_columns] 
DataSet_Test2 = DataSet_Test2_[select_columns] 

DataSet_Train3 = DataSet_Train3_[select_columns] 
DataSet_Test3 = DataSet_Test3_[select_columns] 

DataSet_Train4 = DataSet_Train4_[select_columns] 
DataSet_Test4 = DataSet_Test4_[select_columns] 

y_train1 = DataSet_Train1_['label']
y_test1  = DataSet_Test1_['label']

y_train2 = DataSet_Train2_['label']
y_test2  = DataSet_Test2_['label']

y_train3 = DataSet_Train3_['label']
y_test3  = DataSet_Test3_['label']

y_train4 = DataSet_Train4_['label']
y_test4  = DataSet_Test4_['label']

y_train5 = DataSet_Train5_['label']
y_test5  = DataSet_Test5_['label']

y_train6 = DataSet_Train6_['label']
y_test6  = DataSet_Test6_['label']

y_train7 = DataSet_Train7_['label']
y_test7  = DataSet_Test7_['label']

y_train8 = DataSet_Train8_['label']
y_test8  = DataSet_Test8_['label']

print('--- Weight Anomalo 8 --- \n')
DataSet_Test8_weight_signal =  DataSet_Test8_[DataSet_Test8_['label']==1]['weight'] 
DataSet_Test8_weight_backgr =DataSet_Test8_[DataSet_Test8_['label']==0]['weight'] 
DataSet_TestSM8_weight_signal =  DataSet_Test8_[DataSet_Test8_['label']==2]['weight'] 

DataSet_Train8_weight_signal = DataSet_Train8_[DataSet_Train8_['label']==1]['weight']
DataSet_Train8_weight_backgr = DataSet_Train8_[DataSet_Train8_['label']==0]['weight'] 

DataSet_Test8_weight =  DataSet_Test8_['weight']

print( 'Shape Test Signal', DataSet_Test8_weight_signal.shape)
print( 'Shape Test Background', DataSet_Test8_weight_backgr.shape, '\n')

print('--- Weight Anomalo 7 --- \n')

DataSet_Test7_weight_signal =  DataSet_Test7_[DataSet_Test7_['label']==1]['weight'] 
DataSet_Test7_weight_backgr =  DataSet_Test7_[DataSet_Test7_['label']==0]['weight'] 
DataSet_TestSM7_weight_signal =  DataSet_Test7_[DataSet_Test7_['label']==2]['weight'] 

DataSet_Train7_weight_signal =  DataSet_Train7_[DataSet_Train7_['label']==1]['weight']
DataSet_Train7_weight_backgr =  DataSet_Train7_[DataSet_Train7_['label']==0]['weight'] 

DataSet_Test7_weight =  DataSet_Test7_['weight']

print('--- Weight Anomalo 6 --- \n')

DataSet_Test6_weight_signal =  DataSet_Test6_[DataSet_Test6_['label']==1]['weight'] 
DataSet_Test6_weight_backgr =  DataSet_Test6_[DataSet_Test6_['label']==0]['weight'] 
DataSet_TestSM6_weight_signal =  DataSet_Test6_[DataSet_Test6_['label']==2]['weight'] 

DataSet_Train6_weight_signal =  DataSet_Train6_[DataSet_Train6_['label']==1]['weight']
DataSet_Train6_weight_backgr =  DataSet_Train6_[DataSet_Train6_['label']==0]['weight'] 

DataSet_Test6_weight =  DataSet_Test6_['weight']

print( 'Shape Test Signal',  DataSet_Test7_weight_signal.shape)
print( 'Shape Test Background' , DataSet_Test7_weight_backgr.shape, '\n')

print('--- Weight Anomalo 5 --- \n')

DataSet_Test5_weight_signal =  DataSet_Test5_[DataSet_Test5_['label']==1]['weight'] 
DataSet_Test5_weight_backgr =  DataSet_Test5_[DataSet_Test5_['label']==0]['weight'] 
DataSet_TestSM5_weight_signal =  DataSet_Test5_[DataSet_Test5_['label']==2]['weight'] 

DataSet_Train5_weight_signal =  DataSet_Train5_[DataSet_Train5_['label']==1]['weight']
DataSet_Train5_weight_backgr =  DataSet_Train5_[DataSet_Train5_['label']==0]['weight'] 

DataSet_Test5_weight =  DataSet_Test5_['weight']

print( 'Shape Test Signal',  DataSet_Test7_weight_signal.shape)
print( 'Shape Test Background' , DataSet_Test7_weight_backgr.shape, '\n')

print('--- Weight Anomalo 1 --- \n')

DataSet_Test1_weight_signal =  DataSet_Test1_[DataSet_Test1_['label']==1]['weight'] 
DataSet_Test1_weight_backgr =  DataSet_Test1_[DataSet_Test1_['label']==0]['weight'] 
DataSet_TestSM1_weight_signal =  DataSet_Test1_[DataSet_Test1_['label']==2]['weight'] 

DataSet_Train1_weight_signal =  DataSet_Train1_[DataSet_Train1_['label']==1]['weight']
DataSet_Train1_weight_backgr = DataSet_Train1_[DataSet_Train1_['label']==0]['weight'] 

DataSet_Test1_weight =  DataSet_Test1_['weight']

print( 'Shape Test Signal', DataSet_Test1_weight_signal.shape)
print( 'Shape Test Background' , DataSet_Test1_weight_backgr.shape, '\n')

print('--- Weight Anomalo 2 --- \n')

DataSet_Test2_weight_signal =  DataSet_Test2_[DataSet_Test2_['label']==1]['weight'] 
DataSet_Test2_weight_backgr =  DataSet_Test2_[DataSet_Test2_['label']==0]['weight'] 
DataSet_TestSM2_weight_signal =  DataSet_Test2_[DataSet_Test2_['label']==2]['weight'] 

DataSet_Train2_weight_signal =  DataSet_Train2_[DataSet_Train2_['label']==1]['weight']
DataSet_Train2_weight_backgr = DataSet_Train2_[DataSet_Train2_['label']==0]['weight'] 

DataSet_Test2_weight =  DataSet_Test2_['weight']

print( 'Shape Test Signal', DataSet_Test2_weight_signal.shape)
print( 'Shape Test Background' , DataSet_Test2_weight_backgr.shape, '\n')

print('--- Weight Anomalo 3 --- \n')

DataSet_Test3_weight_signal =  DataSet_Test3_[DataSet_Test3_['label']==1]['weight'] 
DataSet_Test3_weight_backgr =  DataSet_Test3_[DataSet_Test3_['label']==0]['weight'] 
DataSet_TestSM3_weight_signal =  DataSet_Test3_[DataSet_Test3_['label']==2]['weight'] 

DataSet_Train3_weight_signal =  DataSet_Train3_[DataSet_Train3_['label']==1]['weight']
DataSet_Train3_weight_backgr = DataSet_Train3_[DataSet_Train3_['label']==0]['weight'] 

DataSet_Test3_weight =  DataSet_Test3_['weight']

print( 'Shape Test Signal', DataSet_Test3_weight_signal.shape)
print( 'Shape Test Background' , DataSet_Test3_weight_backgr.shape, '\n')

print('--- Weight Anomalo 4 --- \n')

DataSet_Test4_weight_signal = DataSet_Test4_[DataSet_Test4_['label']==1]['weight'] 
DataSet_Test4_weight_backgr = DataSet_Test4_[DataSet_Test4_['label']==0]['weight'] 
DataSet_TestSM4_weight_signal =  DataSet_Test4_[DataSet_Test4_['label']==2]['weight'] 

DataSet_Train4_weight_signal = DataSet_Train4_[DataSet_Train4_['label']==1]['weight']
DataSet_Train4_weight_backgr = DataSet_Train4_[DataSet_Train4_['label']==0]['weight'] 

DataSet_Test4_weight =  DataSet_Test4_['weight']

print( 'Shape Test Signal' , DataSet_Test4_weight_signal.shape)
print( 'Shape Test Background' , DataSet_Test4_weight_backgr.shape)        

Shape for Standard Model -->  (112, 24)
Shape for Anomalo 1 -->  (1544, 24)
Shape for Anomalo 2 -->  (174, 24)
Shape for Anomalo 3 -->  (434, 24)
Shape for Anomalo 4 -->  (742, 24)
Shape for Anomalo 5 -->  (169, 24)
Shape for Anomalo 6 -->  (299, 24)
Shape for Anomalo 7 -->  (672, 24)
Shape for Anomalo 8 -->  (1627, 24)
--- Weight Anomalo 8 --- 

Shape Test Signal (696,)
Shape Test Background (166938,) 

--- Weight Anomalo 7 --- 

--- Weight Anomalo 6 --- 

Shape Test Signal (314,)
Shape Test Background (166938,) 

--- Weight Anomalo 5 --- 

Shape Test Signal (314,)
Shape Test Background (166938,) 

--- Weight Anomalo 1 --- 

Shape Test Signal (662,)
Shape Test Background (166939,) 

--- Weight Anomalo 2 --- 

Shape Test Signal (114,)
Shape Test Background (166939,) 

--- Weight Anomalo 3 --- 

Shape Test Signal (218,)
Shape Test Background (166939,) 

--- Weight Anomalo 4 --- 

Shape Test Signal (342,)
Shape Test Background (166938,)


In [33]:

scaler = StandardScaler()
scaler_prediction = None

# scaler only on the input
X_train_norm1, Y_train_norm1, scaler, scaler_prediction = do_scaler(DataSet_Train1, y_train1, scaler, scaler_prediction)
X_train_norm2, Y_train_norm2, scaler, scaler_prediction = do_scaler(DataSet_Train2, y_train2, scaler, scaler_prediction)
X_train_norm3, Y_train_norm3, scaler, scaler_prediction = do_scaler(DataSet_Train3, y_train3, scaler, scaler_prediction)
X_train_norm4, Y_train_norm4, scaler, scaler_prediction = do_scaler(DataSet_Train4, y_train4, scaler, scaler_prediction)
X_train_norm5, Y_train_norm5, scaler, scaler_prediction = do_scaler(DataSet_Train5, y_train5, scaler, scaler_prediction)
X_train_norm6, Y_train_norm6, scaler, scaler_prediction = do_scaler(DataSet_Train6, y_train6, scaler, scaler_prediction)
X_train_norm7, Y_train_norm7, scaler, scaler_prediction = do_scaler(DataSet_Train7, y_train7, scaler, scaler_prediction)
X_train_norm8, Y_train_norm8, scaler, scaler_prediction = do_scaler(DataSet_Train8, y_train8, scaler, scaler_prediction)

X_test_norm1, Y_test_norm1, scaler, scaler_prediction = do_scaler(DataSet_Test1, y_test1, scaler, scaler_prediction)
X_test_norm2, Y_test_norm2, scaler, scaler_prediction = do_scaler(DataSet_Test2, y_test2, scaler, scaler_prediction)
X_test_norm3, Y_test_norm3, scaler, scaler_prediction = do_scaler(DataSet_Test3, y_test3, scaler, scaler_prediction)
X_test_norm4, Y_test_norm4, scaler, scaler_prediction = do_scaler(DataSet_Test4, y_test4, scaler, scaler_prediction)
X_test_norm5, Y_test_norm5, scaler, scaler_prediction = do_scaler(DataSet_Test5, y_test5, scaler, scaler_prediction)
X_test_norm6, Y_test_norm6, scaler, scaler_prediction = do_scaler(DataSet_Test6, y_test6, scaler, scaler_prediction)
X_test_norm7, Y_test_norm7, scaler, scaler_prediction = do_scaler(DataSet_Test7, y_test7, scaler, scaler_prediction)
X_test_norm8, Y_test_norm8, scaler, scaler_prediction = do_scaler(DataSet_Test8, y_test8, scaler, scaler_prediction)

X_dados_norm = scaler.fit_transform( data_set_dados_multirp )

In [34]:
class_weight1 = { 0: (Y_train_norm1.value_counts()[0] / Y_train_norm1.value_counts()[1]),
                     1: (Y_train_norm1.value_counts()[1] / Y_train_norm1.value_counts()[0])}
class_weight2 = { 0: (Y_train_norm2.value_counts()[0] / Y_train_norm2.value_counts()[1]),
                     1: (Y_train_norm2.value_counts()[1] / Y_train_norm2.value_counts()[0])}
class_weight1 = { 0: (Y_train_norm1.value_counts()[0] / Y_train_norm1.value_counts()[1]),
                     1: (Y_train_norm1.value_counts()[1] / Y_train_norm1.value_counts()[0])}  
class_weight1 = { 0: (Y_train_norm1.value_counts()[0] / Y_train_norm1.value_counts()[1]),
                     1: (Y_train_norm1.value_counts()[1] / Y_train_norm1.value_counts()[0])}
class_weight1 = { 0: (Y_train_norm1.value_counts()[0] / Y_train_norm1.value_counts()[1]),
                     1: (Y_train_norm1.value_counts()[1] / Y_train_norm1.value_counts()[0])}
class_weight1 = { 0: (Y_train_norm1.value_counts()[0] / Y_train_norm1.value_counts()[1]),
                     1: (Y_train_norm1.value_counts()[1] / Y_train_norm1.value_counts()[0])}                                                                                                       

In [35]:

model_number = 4
# Create Model
if model_number == 1:
    model_name = "RF"
    # Random Forest / Best Result - AUC = 0.8663
    param_search = {
        'RF__n_estimators': [100, 200, 300, 400],
        'RF__max_depth': list(range(4,8))

    }
    # create pipeline
    estimators = []
    estimators.append(('RF', RandomForestClassifier()))
    model = Pipeline(estimators)
elif model_number == 2:
    model_name = "ADAB"
    param_search = {"ADAB__base_estimator__max_depth": [1, 2, 3, 4, 5],
                  "ADAB__n_estimators": [700, 800, 900]
                  }

    DTC = DecisionTreeClassifier(max_features="auto", class_weight="balanced", max_depth=None)

    # create pipeline
    estimators = []
    estimators.append(('ADAB', AdaBoostClassifier(base_estimator=DTC)))
    model = Pipeline(estimators)
elif model_number == 3:
    model_name = "XGB"
    param_search = {'XGB__max_depth': [2, 3, 4, 5],
                  'XGB__n_estimators': [200, 400, 600, 800],
                  'XGB__learning_rate': [0.15, 0.2],
                  'XGB__gamma': [0, 0.1]
                  }

    # create pipeline
    estimators = []
    #estimators.append(('standardize', StandardScaler()))
    estimators.append(('XGB', XGBClassifier( booster='gbtree')))
    model = Pipeline(estimators)
elif model_number == 4:
    # Lightgbm
    model_name = "LGBM"
    param_search = {'LGB__num_leaves': [24,128,62],
                 'LGB__n_estimators': [ 1000, 2000],
                 'LGB__learning_rate': [0.0005,0.0001],
                 'LGB__min_child_samples': [100,500],
                 'LGB__colsample_bytree': [ 0.8, 0.9],
                 'LGB__subsample': [ 0.2, 0.3],
                 'LGB__boosting_type': ['gbdt'],
                 'LGB__class_weight': ['balanced', None]
                 }


    # create pipeline
    estimators = []
    #estimators.append(('standardize', StandardScaler()))
    estimators.append(('LGB', LGBMClassifier( max_depth=0 )))
    model = Pipeline(estimators)


In [36]:
from joblib import dump, load


In [37]:
def make_model(X_train_norm,Y_train_norm,param_search,filename):
    n_iter = 8
    cv = 2
    #scoring = make_scorer(fbeta_score, beta=0.5)
    #scoring = 'roc_auc'
    scoring = 'f1'
    #scoring = None
    time_s_ = time.time()
    #search object

    search = GridSearchCV(estimator=model, param_grid=param_search, scoring = scoring ,cv=cv, verbose=1)

    start_time = time.time()

    search_result = search.fit(X_train_norm, Y_train_norm)
    print("--- %s seconds ---" % (time.time() - start_time))
    filename = filename
    dump(search_result, '/content/gdrive/My Drive/Colab Notebooks/output_for_LGBM/Binary_BDT_'+filename+'.joblib')
    # Results
    print("Model: %s Best: %f using %s" % (model_name, search_result.best_score_, search_result.best_params_))
    means = search_result.cv_results_['mean_test_score']
    stds = search_result.cv_results_['std_test_score']
    params = search_result.cv_results_['params']
    time_e_ = time.time()
    print ( "Total time elapsed: {:.0f}".format( time_e_ - time_s_ ) )
    for mean, stdev, param in zip(means, stds, params):
        print("%f (%f) with: %r" % (mean, stdev, param))
    return search_result   

In [38]:
def test_model( X_test, Y_test, X_train, Y_train, model, label, verbose = 1 ):
  
    print(' ---------------- ' + label + '---------------- \n' )
    scaler = StandardScaler()
    X_scaled_test = scaler.fit_transform(X_test)
    X_scaled_train = scaler.fit_transform(X_train)

    y_probs_test = model.predict_proba(X_scaled_test)  # calculate the probability
    prec_test, rec_test, thresh_test = precision_recall_curve(Y_test, y_probs_test[:,1])
    bidx_test = np.argmax(prec_test * rec_test)
    best_cut_test = thresh_test[bidx_test]

    preds_test = y_probs_test[:,1] >= best_cut_test

    y_probs_train = model.predict_proba(X_scaled_train)  # calculate the probability
    prec_train, rec_train, thresh_train = precision_recall_curve(Y_train, y_probs_train[:,1])
    bidx_train = np.argmax(prec_train * rec_train)
    best_cut_train = thresh_train[bidx_train]
    

    preds_train = y_probs_train[:,1] >= best_cut_train

    if verbose == 1:

        fpr_test, tpr_test, thresholds_test = roc_curve(Y_test, y_probs_test[:,1], drop_intermediate=False)
        print('best_cut_test-->', best_cut_test)
        print(f'AUC = {auc(fpr_test, tpr_test)}')
        print("Purity in test sample     : {:2.2f}%".format(100 * precision_score(Y_test, preds_test)))
        print("Efficiency in test sample : {:2.2f}%".format(100 * recall_score(Y_test, preds_test)))
        print("Accuracy in test sample   : {:2.2f}%".format(100 * accuracy_score(Y_test, preds_test)))
        print( "F1_score in test sample  : {:2.2f}%".format(100 * f1_score(Y_test, preds_test)))
        print("\n")
        fpr_train, tpr_train, thresholds_train = roc_curve(Y_train, y_probs_train[:,1], drop_intermediate=False)
        print('best_cut_train-->', best_cut_train)
        print(f'AUC = {auc(fpr_train, tpr_train)}')
        print("Purity in train sample     : {:2.2f}%".format(100 * precision_score(Y_train, preds_train)))
        print("Efficiency in train sample : {:2.2f}%".format(100 * recall_score(Y_train, preds_train)))
        print("Accuracy in train sample   : {:2.2f}%".format(100 * accuracy_score(Y_train, preds_train)))
        print( "F1_score in train sample  : {:2.2f}%".format(100 * f1_score(Y_train, preds_train)))        
    return y_probs_test[:,1], best_cut_test, preds_test


In [39]:
def Volta_Lula( X_train_norm, Y_train_norm, DataSet_Test, y_test, DataSet_Train, y_train,DataSet_Test_weight_signal, DataSet_Test_weight_backgr, label_anomalo, label_save_model ):
    plt.figure(figsize=(14,8))
    search_result = make_model( X_train_norm,Y_train_norm, param_search,label_save_model)
    y_probs,best_cut, preds_test = test_model( DataSet_Test, y_test, DataSet_Train, y_train, search_result, label_anomalo)
    bins = 25

    predict_dados = search_result.predict_proba(np.array(X_dados_norm))[:,1]

    counts_, bin_edges_ = np.histogram( predict_dados[ predict_dados < best_cut], bins = np.linspace(0,1,bins) )
    errors_ = np.sqrt( counts_ )
    bin_centres_ = ( bin_edges_[:-1] + bin_edges_[1:] ) / 2.
    plt.errorbar( bin_centres_, counts_, yerr=errors_, xerr=abs(bin_centres_[1] - bin_centres_[2])/2, fmt='.', label = 'Data-2016', color = 'black' )
    marcia = plt.hist(y_probs[y_test == 0], bins = np.linspace(0,1,bins), weights = DataSet_Test_weight_backgr / test_size, label = 'Background' )
    saco = np.max(marcia[0])
    plt.hist(y_probs[y_test == 1], bins = np.linspace(0,1,bins), histtype = 'step', weights = DataSet_Test_weight_signal / test_size, label = label_anomalo)
    plt.plot( [best_cut,best_cut],[0,saco],  "-.r", linewidth=2.5,  label='Best cut : {:2.3f}'.format( best_cut ) ) 
    plt.legend( loc = "upper right", fontsize = 16 )
    plt.text(best_cut+0.05 , 6, 'Signal Region \n\n Blinded Data', color = 'red', fontsize = 16)    
    plt.yscale('log')

    n_events_back_after_cut = y_probs[ y_test == 0 ] > best_cut
    n_events_signal_after_cut = y_probs[ y_test == 1 ] > best_cut
    n_events_SM_after_cut = y_probs[ y_test == 2 ] > best_cut
    n_eventos_Data_after_cut = predict_dados[ predict_dados >= best_cut ]

    print(' ---------------' +label_anomalo+' --------------- ')
    print('Numero de eventos de background depois do corte -->', DataSet_Test_weight_backgr[ n_events_back_after_cut ].sum() / test_size )
    print('Numero de eventos de signal depois do corte -->', DataSet_Test_weight_signal[ n_events_signal_after_cut].sum() / test_size )
    print('Numero de eventos de dados depois do corte -->', len( n_eventos_Data_after_cut ) )

In [ ]:
Volta_Lula( X_train_norm1, Y_train_norm1, DataSet_Test1, y_test1, DataSet_Train1, y_train1,DataSet_Test1_weight_signal, DataSet_Test1_weight_backgr, r'WWCEP $\alpha_{C}^{W}/\Lambda^{2} = 2.0 \times 10^{-6} $', 'anomalo1_SM_como_signal_cut_Mww600' )

Fitting 2 folds for each of 192 candidates, totalling 384 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


In [ ]:
Volta_Lula( X_train_norm2, Y_train_norm2, DataSet_Test2, y_test2, DataSet_Train2, y_train2,DataSet_Test2_weight_signal, DataSet_Test2_weight_backgr, r'WWCEP $\alpha_{C}^{W}/\Lambda^{2} = 2.0 \times 10^{-6} $', 'anomalo2_SM_como_signalcut_Mww600' )

In [ ]:
Volta_Lula( X_train_norm3, Y_train_norm3, DataSet_Test3, y_test3, DataSet_Train3, y_train3,DataSet_Test3_weight_signal, DataSet_Test3_weight_backgr, r'WWCEP $\alpha_{C}^{W}/\Lambda^{2} = 2.0 \times 10^{-6} $', 'anomalo3_SM_como_signalcut_Mww600' )

In [ ]:
Volta_Lula( X_train_norm4, Y_train_norm4, DataSet_Test4, y_test4, DataSet_Train4, y_train4,DataSet_Test4_weight_signal, DataSet_Test4_weight_backgr, r'WWCEP $\alpha_{C}^{W}/\Lambda^{2}=8.0 \times 10^{-6}$', 'anomalo4_SM_como_signalcut_Mww600' )

In [ ]:
Volta_Lula( X_train_norm5, Y_train_norm5, DataSet_Test5, y_test5, DataSet_Train5, y_train5,DataSet_Test5_weight_signal, DataSet_Test5_weight_backgr, r'WWCEP $\alpha_{0}^{W}/\Lambda^{2}=0.5 \times 10^{-5}$', 'anomalo5_SM_como_signalcut_Mww600' )

In [ ]:
Volta_Lula( X_train_norm6, Y_train_norm6, DataSet_Test6, y_test6, DataSet_Train6, y_train6,DataSet_Test6_weight_signal, DataSet_Test6_weight_backgr, r'WWCEP $\alpha_{0}^{W}/\Lambda^{2}=1.0 \times 10^{-6}$', 'anomalo6_SM_como_signalcut_Mww600' )

In [ ]:
Volta_Lula( X_train_norm7, Y_train_norm7, DataSet_Test7, y_test7, DataSet_Train7, y_train7,DataSet_Test7_weight_signal, DataSet_Test7_weight_backgr, r'WWCEP $\alpha_{0}^{W}/\Lambda^{2}=2.0 \times 10^{-6}$', 'anomalo7_SM_como_signalcut_Mww600' )

In [ ]:
Volta_Lula( X_train_norm8, Y_train_norm8, DataSet_Test8, y_test8, DataSet_Train8, y_train8,DataSet_Test8_weight_signal, DataSet_Test8_weight_backgr, r'WWCEP $\alpha_{0}^{W}/\Lambda^{2}=5.0 \times 10^{-6}$', 'anomalo8_SM_como_signalcut_Mww600' )